<a href="https://colab.research.google.com/github/adrianwalsh1990/crystal_structure_prediction/blob/main/Crystal_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the google colab notebook for predicting the pointgroups and Bravais lattice of a set of atomic position. 

Set x equal to the list of atomic positions.


In [ ]:
x = [[
      [0.0000000000000,	0.00000000000000,	0.0000000000000000],
      [1.41159734216321632,	4.23479202648963,	4.23479202648963],
      [0.00000000000000,	2.82319468432642,	2.82319468432642],
      [1.41159734216321632,	1.41159734216321632,	1.41159734216321632],
      [2.82319468432642,	0.0000000000000000,	2.82319468432642],
      [4.23479202648963,	4.23479202648963,	1.41159734216321632],
      [2.82319468432642,	2.82319468432642,	0.0000000000000000],
      [4.23479202648963,	1.41159734216321632,	4.23479202648963]
      ]]

Please enter the crystal system:

Choose from 'Triclinic', 'Monoclinic', 'Orthorhombic', 'Tetragonal', 'Trigonal', 'Hexagonal', 'Cubic'

In [ ]:
CrySys = 'Cubic'

Run the following cells

# Preamble

For running in Google Colab, determine the path to the folder in google drive, or clone the repository.

In [ ]:
#@title Paths and Filenames

# # Optional mount and work with google drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# path = '/content/gdrive/MyDrive/Colab Notebooks/StructurePrediction/'
# weights_path = path + 'Weights/'
# data_path = path + 'DataSets/'


# For use with Google Colab without mounting drive
%cd /content
!rm -rf crystal_structure_prediction
!git clone --single-branch --depth=1 https://github.com/adrianwalsh1990/crystal_structure_prediction.git
%cd crystal_structure_prediction
!unzip '/content/crystal_structure_prediction/DataSets/materialsdatabase.json.zip' -d '/content/crystal_structure_prediction/DataSets/'

weights_path = '/content/crystal_structure_prediction/Weights/'
data_path = '/content/crystal_structure_prediction/DataSets/'
filename = 'materialsdatabase.json'
fname = data_path + filename 



/content
Cloning into 'crystal_structure_prediction'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 95 (delta 38), reused 93 (delta 38), pack-reused 0
Unpacking objects: 100% (95/95), done.
Checking out files: 100% (91/91), done.
/content/crystal_structure_prediction
Archive:  /content/crystal_structure_prediction/DataSets/materialsdatabase.json.zip
  inflating: /content/crystal_structure_prediction/DataSets/materialsdatabase.json  


In [ ]:
#@title Import libraries and  mount drive
!pip install trimesh
import pandas as pd
import trimesh
import numpy as np
import tensorflow as tf
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.activations import relu
from tensorflow.python.keras.backend import categorical_crossentropy
from tensorflow.python.keras.metrics import accuracy
from tensorflow.python.keras.metrics import AUC 
from tensorflow.python.keras.metrics import FalsePositives
from tensorflow.python.keras.metrics import FalseNegatives
from tensorflow.python.keras.metrics import TruePositives
from tensorflow.python.keras.metrics import TrueNegatives
from tensorflow.python.keras.optimizer_v1 import adam
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt



# Prepare the data

In [ ]:
#@title Padding of the sites to match the input size
file = pd.read_json(fname)

# Max number of atomic sites
num_sites = file['nsites'].to_numpy()
NUM_POINTS = np.amax(num_sites)


# Pad the atomic positions until all have the same number of sites
x_padded = np.array([xi + [xi[0]] * (NUM_POINTS - len(xi)) for xi in x])

Create the PointGroup model, that matches the set of weights from the training process.

# Prediction of the point groups

In [ ]:
# @title Create the model

# Example modified from the Keras Code Example 
# https://keras.io/examples/vision/pointnet/

METRIC = "TruePositives","TrueNegatives","FalsePositives","FalseNegatives"


def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

def tnet(inputs, num_features):
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)

x = layers.GlobalMaxPooling1D()(x)


x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)


outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=[METRIC],
)

Prediction of the point groups

In [ ]:
# Identify the necessary point groups
PointGroups = {
    'Triclinic': ['1bar'],
    'Monoclinic': ['2','m','2m'],
    'Orthorhomic':['222','mm2','mmm'],
    'Tetragonal': ['4','4bar','4m','422','4mm','4bar2m','4mmm'],
    'Trigonal': ['3','3bar','32','3m','3barm'],
    'Hexagonal': ['6','6bar','6m','622','6mm','6barm2','6mmm'],
    'Cubic':['23','2m3bar','432','4bar3m','4m3bar2m']
}

PGS = PointGroups.get(CrySys,-1)

# For each point group predict whether that symmetry is present (indicated by a 1) 
# or not(indicated by a 0), by loading the correct weights for the model, and
# making a prediction from the input points

predictions = []
for PG in PGS:

  model_name = weights_path + 'Pointgroup-' + PG
  model.load_weights(model_name)

  preds = model.predict(x_padded)
  preds[preds < 0.5] = 0
  preds[preds >= 0.5] = 1

  predictions.append(preds[0])
  print(PG,' - ',preds[0]) 

23  -  [1.]
2m3bar  -  [0.]
432  -  [1.]
4bar3m  -  [1.]
4m3bar2m  -  [1.]


# Prediction of the Bravais Lattices

Create the Bravais lattice model, that matches the set of weights from the training process.

In [ ]:
# @title Create the BRAVIS model

# If the crsytal system is triclinic or hexagonal the notebook stops here as only
# the primitive lattice is present
if CrySys == 'Triclinic' or CrySys == 'Hexagonal':
  !kill -9 -1


# Depending on the crystal system certain lattices are possible
Bravais = {    
    'Monoclinic': ['P','C'],
    'Orthorhomic':['P','I','F','A','C'],
    'Tetragonal': ['P','I'],
    'Trigonal': ['P','R'],
    'Cubic':['P','I','F']
}

NUM_CLASSES = len(Bravais.get(CrySys,-1))

def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

def tnet(inputs, num_features):
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)


x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
# model.summary()

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["sparse_categorical_accuracy"],
)

Prediction of the Bravais Lattice

In [ ]:
# load the correct Bravais lattice model
model_name = weights_path + 'BRAVIS_' + CrySys
model.load_weights(model_name)


preds = model.predict(x_padded)

preds[preds < 0.5] = 0
preds[preds >= 0.5] = 1

BL = Bravais.get(CrySys,-1)[np.where(preds[0] == np.amax(preds[0]))[0][0]]

print('Bravais Lattice =', BL)



Bravais Lattice = F


# Output of the predictions

In [ ]:
# @title Output:
print('Bravais Lattice =', BL)

for i in range(len(PGS)):
  print(PGS[i],'\t',predictions[i])

Bravais Lattice = F
23 	 [1.]
2m3bar 	 [0.]
432 	 [1.]
4bar3m 	 [1.]
4m3bar2m 	 [1.]
